Say there is a coin that before a long game is set one of two states, state 1 $p_H=1/2$ or state 2  $p_H = 3/4$ . If you do not know to what state the coin was set before the game but believe that it was set to state 2 with probability $0.9$, how do coin tosses update your belief?


In [3]:
a=142857

In [4]:
1/7

0.14285714285714285